# Secretária de Segurança Pública - Ocorrências Criminais Mapa de Calor

In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Coordenadas para o centro de São José dos Campos - SP
centro_sjc = [-23.2237, -45.9009]

# Carregar os três arquivos CSV
file_path_2022 = '../data/SJCDadosCriminais_2022.csv'
file_path_2023 = '../data/SJCDadosCriminais_2023.csv'
file_path_2024 = '../data/SJCDadosCriminais_2024.csv'

data_2022 = pd.read_csv(file_path_2022)
data_2023 = pd.read_csv(file_path_2023)
data_2024 = pd.read_csv(file_path_2024)

# Concatenar os três DataFrames
data = pd.concat([data_2022, data_2023, data_2024], ignore_index=True)

# Obter todos os tipos de crimes únicos na coluna NATUREZA_APURADA
natureza_apurada_unique = sorted(data['NATUREZA_APURADA'].unique())

# Obter todos os anos únicos na coluna ANO_ESTATISTICA e adicionar a opção "TODOS"
anos_unique = ['TODOS'] + sorted(data['ANO_ESTATISTICA'].unique())

# Criar dropdowns para selecionar o tipo de crime e o ano
crime_type_dropdown = widgets.Dropdown(
    options=natureza_apurada_unique,
    description='Tipo de Crime:',
    disabled=False,
)

ano_dropdown = widgets.Dropdown(
    options=anos_unique,
    description='Ano:',
    disabled=False,
)

# Função para gerar o mapa de mancha criminal com base no tipo de crime e ano selecionado
def generate_crime_map(crime_type, ano):
    clear_output(wait=True)  # Limpa a saída anterior
    display(crime_type_dropdown, ano_dropdown, generate_button)  # Redesenha os widgets
    
    # Filtrar dados com base no tipo de crime e ano selecionado
    if ano == 'TODOS':
        data_filtered = data[data['NATUREZA_APURADA'] == crime_type]
    else:
        data_filtered = data[(data['NATUREZA_APURADA'] == crime_type) & (data['ANO_ESTATISTICA'] == ano)]
    
    # Filtrar apenas os dados com coordenadas válidas (para o mapa)
    data_filtered_map = data_filtered[data_filtered['LATITUDE'].notnull() & data_filtered['LONGITUDE'].notnull()]
    
    # Criar o mapa centrado em São José dos Campos - SP
    crime_map = folium.Map(location=centro_sjc, zoom_start=12)
    
    # Adicionar a mancha criminal usando HeatMap (apenas para dados com coordenadas)
    heat_data = [[row['LATITUDE'], row['LONGITUDE']] for index, row in data_filtered_map.iterrows()]
    HeatMap(heat_data).add_to(crime_map)
    
    # Adicionar uma legenda ao mapa
    legend_html = """
    <div style="position: fixed; 
         bottom: 50px; left: 50px; width: 200px; height: 150px; 
         border:2px solid grey; z-index:9999; font-size:14px;
         background-color:white;
         ">
         &nbsp;<b>Legenda</b> <br>
         &nbsp;<i style="color: blue;">●</i> Baixa Densidade <br>
         &nbsp;<i style="color: green;">●</i> Média-Baixa Densidade <br>
         &nbsp;<i style="color: yellow;">●</i> Média Densidade <br>
         &nbsp;<i style="color: orange;">●</i> Média-Alta Densidade <br>
         &nbsp;<i style="color: red;">●</i> Alta Densidade <br>
    </div>
    """
    crime_map.get_root().html.add_child(folium.Element(legend_html))
    
    # Exibir estatísticas (incluindo dados sem coordenadas)
    num_eventos = data_filtered.shape[0]
    
    # Contagem de incidências por bairro
    top_bairros = data_filtered['BAIRRO'].value_counts().head(10)
    top_bairros_html = "<br>".join([f"{bairro}: {count} ocorrências" for bairro, count in top_bairros.items()])
    
    estatisticas_html = f"""
    <p><b>Estatísticas para {crime_type} em {ano}:</b><br>
    Total de Eventos: {num_eventos}<br><br>
    <b>Top 5 Bairros com Mais Incidências:</b><br>{top_bairros_html}</p>
    """
    display(HTML(estatisticas_html))
    
    # Salvar e exibir o mapa
    crime_map.save('../export/crime_heat_map.html')
    display(crime_map)

# Criar um botão para gerar o mapa
generate_button = widgets.Button(description="Gerar Mapa")

# Conectar o botão à função
generate_button.on_click(lambda x: generate_crime_map(crime_type_dropdown.value, ano_dropdown.value))

# Exibir widgets
display(crime_type_dropdown)
display(ano_dropdown)
display(generate_button)


Dropdown(description='Tipo de Crime:', index=2, options=('APREENSÃO DE ENTORPECENTES', 'ESTUPRO', 'ESTUPRO DE …

Dropdown(description='Ano:', options=('TODOS', 2022, 2023, 2024), value='TODOS')

Button(description='Gerar Mapa', style=ButtonStyle())